#  Install, Setup gdrive and imports

In [9]:
import matplotlib.pyplot as plt
import glob

from tqdm.autonotebook import tqdm
import cv2
import os
import numpy as np
from skimage import color,img_as_ubyte,img_as_bool
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches
from datetime import datetime
from PIL import Image
import pandas as pd

from skimage.morphology import remove_small_objects, remove_small_holes, disk
from skimage.morphology import binary_erosion, binary_dilation


In [ ]:
IN_CWR_SERVER = False 
slideset = 'cav2'

# Files


In [ ]:
if IN_CWR_SERVER : # CWR server
  datadir = "/directory_in_CWR_server/"
else : # LOCAL
  datadir = "C:\\research\\cav\\datamvt\\"

# Input Folders and Files
tilesdir = datadir + 'tiles\\' + slideset + '\\'

#output folder and files
outdir = datadir + "output\\" + slideset + "\\"
classdir=datadir + 'classes\\' + slideset + "\\" 

#Configs
save_mask = False
save_ppt = False
save_csv = False

# Functions

In [ ]:
def newppt (subtitle) :
    title = "MOVAT Segmentation"
    date = datetime.today()
    author = " Samhith/Abby"
    comments = subtitle
    stain = "Movat"


    prs = Presentation()
    prs.slide_width = Inches(20)
    prs.slide_height = Inches(10)
    blank_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(blank_slide_layout)
    slide.placeholders[0].text = title

    tf = slide.placeholders[1].text_frame
    tf.text = f'Date: {date}\n'
    tf.text += f"Author: {author}\n"
    tf.text += f"Reference: {comments}\n"
    tf.text += f"Stain: {stain}\n"
    return prs



In [ ]:

#%%time
#b_hmask = ioh[:,:,2]<80
BLACK_MIN = np.array([1,1,1],np.uint8)
BLACK_MAX = np.array([255,255,70],np.uint8)
# W
WHITE_MIN = np.array([0, 0, 180],np.uint8) 
WHITE_MAX = np.array([255,60, 255],np.uint8)    
# Y
YELLOW_MIN = np.array([5, 60, 70],np.uint8) 
YELLOW_MAX = np.array([60, 255, 255],np.uint8)    
# BLUE
BLUE_MIN = np.array([60, 60, 70],np.uint8)
BLUE_MAX = np.array([140, 255, 255],np.uint8)    
# RED
L_RED_MIN = np.array([0,60,70],np.uint8)
L_RED_MAX = np.array([5,255,255],np.uint8) 
U_RED_MIN = np.array([140,60,70],np.uint8)
U_RED_MAX = np.array([180,255,255],np.uint8)

SELEM = disk(3) # shape = 7,7 (core is 5,5 matching kernel)
KERNEL = (5,5)

#helper function to blend two images
def blend2Images(img, mask):
    if (img.ndim == 3):
        img = color.rgb2gray(img)
    if (mask.ndim == 3):
        mask = color.rgb2gray(mask)
    img = img[:, :, None] * 1.0  # can't use boolean
    mask = mask[:, :, None] * 1.0
    out = np.concatenate((mask, img, mask), 2) * 255
    return out.astype('uint8')

def addimagetoslide(slide,img,left,top, height, width, resize = 0.25):
    res = cv2.resize(img , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC) 
    image_stream = BytesIO()
    Image.fromarray(res).save(image_stream,format="PNG")
    pic = slide.shapes.add_picture(image_stream, left, top ,height,width)
    image_stream.close()   

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def finalProcessing_D(img_mask):
    mask_spur =img_mask
    mask_spur = remove_small_objects(img_as_bool(mask_spur), min_size=100) ###=== 15
    mask_spur = remove_small_holes(mask_spur, 5000) 
    return img_as_ubyte(mask_spur)

def finalProcessing_W(img_mask):
    prev_mask =img_mask
    mask_spur = binary_dilation(img_as_bool(prev_mask), SELEM)
    mask_spur = remove_small_objects(mask_spur, min_size=100)
    mask_spur = ~remove_small_objects(~mask_spur, min_size=100)
    mask_spur = remove_small_holes(mask_spur, 5000)

    mask_spur = ~cv2.subtract(img_as_ubyte(~mask_spur),img_as_ubyte(prev_mask))
    return mask_spur

def finalProcessing_B(img_mask):
    prev_mask =img_mask
    mask_spur = binary_dilation(img_as_bool(prev_mask), SELEM)
    mask_spur = remove_small_objects(mask_spur, min_size=100)
    mask_spur = remove_small_holes(mask_spur, 100)

    return img_as_ubyte(mask_spur)

def finalProcessing_Y(img_mask):
    prev_mask =img_mask
    mask_spur = binary_dilation(img_as_bool(prev_mask), SELEM)
    mask_spur = remove_small_objects(mask_spur, min_size=100)
    mask_spur = ~remove_small_objects(~mask_spur, min_size=100)
    mask_spur = remove_small_holes(mask_spur, 5000)

    mask_spur = ~cv2.subtract(img_as_ubyte(~mask_spur),img_as_ubyte(prev_mask))
    return mask_spur

def finalProcessing_R(img_mask_use):
    prev_mask =img_mask_use
    mask_spur = binary_erosion(img_as_bool(prev_mask), SELEM)
    mask_spur = remove_small_objects(mask_spur, min_size=400)
    mask_spur = remove_small_holes(mask_spur, 400) 

    return img_as_ubyte(mask_spur) 

def finalProcessing_mcar(img_mask_use):
    # dilation + fill large holes + erosion
    prev_mask =img_mask_use
    mask_spur = binary_dilation(img_as_bool(prev_mask), disk(30))
    mask_spur = remove_small_holes(mask_spur, 5000) 
    mask_spur = remove_small_objects(mask_spur, min_size=5000)
    mask_spur = binary_erosion(mask_spur, disk(20)) # ~50% of dilation

    return img_as_ubyte(mask_spur) 

def histAdjustRedChannel(bgr_image):
    B,G,R = cv2.split(bgr_image) #get single 8-bits channel
    ER=cv2.equalizeHist(R)
    equal_r=cv2.merge((B,G,ER))  #merge it back
    return equal_r
    
def saveClass(tile, oi, dm, wm, bm, ym, rm, mcarm, strom, ymcarm, bmcarm, wmcarm, overstained=False, resize = .25):
    d = cv2.resize(dm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    w = cv2.resize(wm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    b = cv2.resize(bm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    y = cv2.resize(ym , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    r = cv2.resize(rm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    mcar = cv2.resize(mcarm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    s = cv2.resize(strom , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    ymcar = cv2.resize(ymcarm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    bmcar = cv2.resize(bmcarm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)
    wmcar = cv2.resize(wmcarm , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC)

    if (overstained) :
        cv2.imwrite(classdir + os.sep + tile + "_bright.png", oi)
    cv2.imwrite(classdir + os.sep + tile + "_black.png", d)
    cv2.imwrite(classdir + os.sep + tile + "_white.png", w)
    cv2.imwrite(classdir + os.sep + tile + "_blue.png", b)
    cv2.imwrite(classdir + os.sep + tile + "_yellow.png", y)
    cv2.imwrite(classdir + os.sep + tile + "_red.png", r)    
    cv2.imwrite(classdir + os.sep + tile + "_all_myo.png", mcar)    
    cv2.imwrite(classdir + os.sep + tile + "_stro.png", s)    
    cv2.imwrite(classdir + os.sep + tile + "_y_myo.png", ymcar)    
    cv2.imwrite(classdir + os.sep + tile + "_b_myo.png", bmcar)    
    cv2.imwrite(classdir + os.sep + tile + "_w_myo.png", wmcar)    

def addSlide(prs, tn, oi, oic, dm, wm, bm, ym, rm, mcarm, strom, ymcarm, bmcarm, wmcarm, overstained=False):
    ##add a new slide for this set of images 
    blank_slide_layout = prs.slide_layouts[6]
    slide1 = prs.slides.add_slide(blank_slide_layout)
    # s1, row 1
    addimagetoslide(slide1, oi, Inches(0),Inches(0),Inches(5),Inches(5)) # org
    addimagetoslide(slide1, rm, Inches(5),Inches(0),Inches(5),Inches(5)) 
    addimagetoslide(slide1, ym, Inches(10),Inches(0),Inches(5),Inches(5))
    addimagetoslide(slide1, bm, Inches(15),Inches(0),Inches(5),Inches(5))
    # s1, row 2
    if (overstained) :
        addimagetoslide(slide1, oic, Inches(0),Inches(5),Inches(5),Inches(5)) # bright image
    txBox = slide1.shapes.add_textbox(Inches(0), Inches(5),Inches(4),Inches(4) )
    addimagetoslide(slide1, mcarm, Inches(5),Inches(5),Inches(5),Inches(5))
    addimagetoslide(slide1, dm, Inches(10),Inches(5),Inches(5),Inches(5))
    addimagetoslide(slide1, wm, Inches(15),Inches(5),Inches(5),Inches(5))

    tf = txBox.text_frame
    tf.text = f"{tn}\n"
    tf.text += f"orig | red   | yellow | blue\n"
    if (overstained) :
        tf.text += f"bright | myocar | black | white"
    else :    
        tf.text += f"text | myocar | black | white"

    slide2 = prs.slides.add_slide(blank_slide_layout)
    # s2, row 1
    addimagetoslide(slide2, mcarm, Inches(0),Inches(0),Inches(5),Inches(5)) # org image
    addimagetoslide(slide2, blend2Images(img_as_bool(mcarm), img_as_bool(strom)), Inches(5),Inches(0),Inches(5),Inches(5))
    addimagetoslide(slide2, blend2Images(img_as_bool(mcarm), img_as_bool(ymcarm)), Inches(10),Inches(0),Inches(5),Inches(5))
    addimagetoslide(slide2, blend2Images(img_as_bool(mcarm), img_as_bool(bmcarm)), Inches(15),Inches(0),Inches(5),Inches(5))

    # s2, row 2
    txBox = slide2.shapes.add_textbox(Inches(0), Inches(5),Inches(4),Inches(4) )
    addimagetoslide(slide2, strom, Inches(5),Inches(5),Inches(5),Inches(5))
    addimagetoslide(slide2, ymcarm, Inches(10),Inches(5),Inches(5),Inches(5))
    #addimagetoslide(slide2, bmcarm, Inches(15),Inches(5),Inches(5),Inches(5))
    addimagetoslide(slide2, blend2Images(img_as_bool(mcarm), img_as_bool(wmcarm)), Inches(15),Inches(5),Inches(5),Inches(5))
    tf = txBox.text_frame
    tf.text = f"{tn}\n"
    tf.text += f"myocar| stroma/mycar| Y /myocar | Blue/myocar \n"
    tf.text += f"text  | stroma      | y-myocar | W /myocar"

    

In [ ]:
def segment(bgr_img, overstained=False):
    if (overstained) :
        ############## Bright up the image
        alpha = 1.5 # Contrast control (1.0-3.0)
        beta = 0 # Brightness control (0-100)
        bgr_img = cv2.convertScaleAbs(bgr_img, alpha=alpha, beta=beta)
    dst=cv2.GaussianBlur(bgr_img,KERNEL,cv2.BORDER_DEFAULT)
    io=cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    ioh=cv2.cvtColor(io,cv2.COLOR_RGB2HSV)
    ## BLACK-----------------------
    d_mask = cv2.inRange(ioh, BLACK_MIN, BLACK_MAX)
    d_mask = finalProcessing_D(d_mask)
    ## WHITE-----------------------
    w_mask = cv2.inRange(ioh, WHITE_MIN, WHITE_MAX)
    w_mask = finalProcessing_W(w_mask)
    ## BLUE-----------------------
    b_mask = cv2.inRange(ioh, BLUE_MIN, BLUE_MAX)
    b_mask = finalProcessing_B(b_mask)
    ## YELLOW-----------------------
    y_mask = cv2.inRange(ioh, YELLOW_MIN, YELLOW_MAX)
    y_mask = finalProcessing_Y(y_mask)
    y_mask = cv2.subtract(y_mask,d_mask + w_mask + b_mask) #  
    ## RED RED-----------------------
    # Range for lower & uppper red
    r_mask1 = cv2.inRange(ioh, L_RED_MIN, L_RED_MAX)
    r_mask2 = cv2.inRange(ioh, U_RED_MIN, U_RED_MAX)
    r_mask = r_mask1 + r_mask2
    r_mask = finalProcessing_R(r_mask)
    r_mask = cv2.subtract(r_mask, d_mask + w_mask + b_mask + y_mask) #  
    ## Myocardium RED-----------------------
    mcar_mask = finalProcessing_mcar(r_mask)

    return io, d_mask, w_mask, b_mask, y_mask, r_mask, mcar_mask

# Main

In [ ]:
COLUMNS=['cohort', 'fname', 'tile','black', 'white', 'blue', 'collagen', 'myocyte', 'myocar', 'stro_myocar', 'collagen_myocar', 'blue_myocar', "white_myocar"]
def movatmain(coho, fpattern, overstained=False) :
    if overstained :
        #img_files = glob.glob(tilesdir + '*/OS' + fpattern)[:2] #for tests just pick up 2 files
        img_files = glob.glob(tilesdir + '*/OS' + fpattern)
        out_fname = '11_mvt_' + coho + '_OS'
    else :
        #img_files = glob.glob(tilesdir + '*'  + fpattern)[:2] #for tests just pick up 2 files
        img_files = glob.glob(tilesdir + '*'  + fpattern)
        out_fname = '11_mvt_' + coho
    if (save_ppt) :
        prs = newppt(out_fname)
    df=pd.DataFrame()
    for image_fname in tqdm(img_files):
        tile = os.path.basename(image_fname).replace('.png','')
        fname = tile.split("_", 1)[0]
        cohort = fname.replace(slideset.upper() + "-", "")[:2]

    ################# SEGMENTATION #############
        bgr_i = cv2.imread(image_fname)
        rgb_i = cv2.cvtColor(bgr_i, cv2.COLOR_BGR2RGB)
        if (overstained) :
            oi, d_mask, w_mask, b_mask, y_mask, r_mask, mcar_mask = segment(bgr_i, overstained=True)
        else :
            oi, d_mask, w_mask, b_mask, y_mask, r_mask, mcar_mask = segment(bgr_i, overstained=False)
        stro_mask = cv2.subtract(mcar_mask, r_mask)
        y_mcar_mask = cv2.subtract(y_mask, ~mcar_mask)
        b_mcar_mask = cv2.subtract(b_mask, ~mcar_mask)    
        w_mcar_mask = cv2.subtract(w_mask, ~mcar_mask)    
    ################# CALC AREA #############
        d_area = np.sum(d_mask == 255)
        w_area = np.sum(w_mask == 255)
        b_area = np.sum(b_mask == 255)
        y_area = np.sum(y_mask == 255)
        r_area = np.sum(r_mask == 255)
        mcar_area = np.sum(mcar_mask == 255)
        s_area = np.sum(stro_mask == 255)
        y_mcar_area = np.sum(y_mcar_mask == 255)
        b_mcar_area = np.sum(b_mcar_mask == 255)
        w_mcar_area = np.sum(w_mcar_mask == 255)

        temp_df = pd.DataFrame([[cohort, fname, tile, d_area, w_area, b_area, y_area, r_area, mcar_area, s_area, y_mcar_area, b_mcar_area, w_mcar_area]], columns=COLUMNS)
    ################# SAVE #############
        if (save_mask) :
            if (overstained) :
                saveClass(tile, oi, d_mask, w_mask, b_mask, y_mask, r_mask, mcar_mask, stro_mask, y_mcar_mask, b_mcar_mask, w_mcar_mask, overstained=True)
            else :
                saveClass(tile, None, d_mask, w_mask, b_mask, y_mask, r_mask, mcar_mask, stro_mask, y_mcar_mask, b_mcar_mask, w_mcar_mask, overstained=False)    
        if (save_ppt) :
            if (overstained) :
                addSlide(prs, tile, rgb_i, oi, d_mask, w_mask, b_mask, y_mask, r_mask, mcar_mask, stro_mask, y_mcar_mask, b_mcar_mask, w_mcar_mask, overstained=True)
            else :
                addSlide(prs, tile, rgb_i, None, d_mask, w_mask, b_mask, y_mask, r_mask, mcar_mask, stro_mask, y_mcar_mask, b_mcar_mask, w_mcar_mask, overstained=False)    

        df = df.append(temp_df)
        #break
    if (save_ppt) :
        pptxfname =outdir + out_fname + ".pptx"
        prs.save(pptxfname)
    # All done, save CSV    
    if (save_csv) :
        #df.to_csv(outdir + out_fname + '.csv',index=False)
        df.to_csv(f"{outdir}{out_fname}.csv.zip", index=False, compression=dict(method='zip', archive_name=f'{out_fname}.csv'))
    ######################################################################################################
    print(f'Segmented {len(img_files)} files. Output saved in {out_fname}.')

In [ ]:
#imgdir=tilesdir + '*/OS' # for OS
dc_fpattern="\\" + slideset + "-DC-2*.png"
hc_fpattern="\\" + slideset + "-HC-2*.png"
dy_fpattern="\\" + slideset + "-DY1-4*.png"
us_fpattern="\\" + slideset + "-USHER-*.png"
ma_fpattern="\\" + slideset + "-MATCH-*.png"

movatmain('DC', dc_fpattern, overstained=True)
movatmain('DC', dc_fpattern, overstained=False)
#movatmain('HC', hc_fpattern, overstained=True)
#movatmain('HC', hc_fpattern, overstained=False)
##movatmain('DY', dy_fpattern, overstained=True)
##movatmain('DY', dy_fpattern, overstained=False)
# movatmain('US', us_fpattern, overstained=True)
# movatmain('US', us_fpattern, overstained=False)
# movatmain('MA', ma_fpattern, overstained=True)
# movatmain('MA', ma_fpattern, overstained=False)

